In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pyautogui
import time
import glob
import shutil

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
download_dir = os.path.expanduser("/Users/Desktop/") # Replace with your own local path
def downloadRatio(idrssd_list, driver):
    for idrssd in idrssd_list:
        try:
            # Step 1: Go back to FFIEC main page (reset the form)
            driver.get("https://cdr.ffiec.gov/public/ManageFacsimiles.aspx") ## FFIEC website

            # Step 2: Select "Uniform Bank Performance Report (UBPR)" from the dropdown
            # Wait for the dropdown to be present and clickable
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.ID, "reportTypeDropDownList"))
            )
            WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.ID, "reportTypeDropDownList"))
            )
            print('ok')
            report_type_dropdown = Select(driver.find_element(By.ID, "reportTypeDropDownList"))
            report_type_dropdown.select_by_value("283")

            # Step 3: Enter the IDRSSD
            idrssd_input = driver.find_element(By.ID, "txtUbprEsrUniqueIdentifier")
            idrssd_input.clear()
            idrssd_input.send_keys(idrssd) ## 

            # Step 4: Click "Generate Report"
            driver.find_element(By.ID, "btnGenerateByIdentifierEsrUbpr").click()
            time.sleep(3)
            WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.ID, "btnGenerateReportSrf"))
            ).click()

            # Step 5: Switch to new window
            WebDriverWait(driver, 10).until(EC.new_window_is_opened)
            new_window = [w for w in driver.window_handles if w != driver.current_window_handle][0]
            driver.switch_to.window(new_window)
            time.sleep(5)

            # Step 6: Click "Summary Ratios"
            summary_ratio_link = driver.find_element(By.XPATH, "//*[@id='tc-link^502574^Summary Ratios--Page 1^Summary Ratios']") # Enter Summary Ratio
            driver.execute_script("arguments[0].click();", summary_ratio_link)
            time.sleep(6)

            # Step 7: Open chart
            #chart_button = driver.find_element(By.XPATH, "//*[@id='trend-dl-39-644066-dl']")
            # Here we are aiming the Net Loans & Leases to Assets Liguidity Ratio; Replace with your target ratio
            chart_button = driver.find_element(By.XPATH, "//*[@id='trend-dl-40-609723-dl']") 
            driver.execute_script("arguments[0].click();", chart_button)
            time.sleep(6)

            # Step 8: Download CSV using pyautogui
            x, y = 1376, 307
            pyautogui.moveTo(x, y)
            time.sleep(5)
            pyautogui.click()
            time.sleep(1)

            pyautogui.moveTo(x, y + 120)
            time.sleep(5)
            pyautogui.click()
            print(f"Download triggered for {idrssd}")

            # Step 9: Wait for file and rename
            time.sleep(13)  # Let it finish downloading
            csv_files = glob.glob(os.path.join(download_dir, "*.csv"))
            if csv_files:
                latest_file = max(csv_files, key=os.path.getctime)
                new_name = os.path.join(download_dir, f"{idrssd}.csv")
                shutil.move(latest_file, new_name)
                print(f"Saved as {new_name}")
                idrssd_list.remove(idrssd)
            else:
                print(f"No CSV downloaded for {idrssd}")

            # Step 10: Close popup tab and switch back
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)

        except Exception as e:
            print(f"Error processing IDRSSD {idrssd}: {e}")
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            continue
    # Finish
    driver.quit()




In [ ]:
# Code with whole list
# Initialize the WebDriver
# driver = webdriver.Chrome()  # Or webdriver.Firefox() if using Firefox


# Make sure the download folder exists
os.makedirs(download_dir, exist_ok=True)
idrssd_list = ['271136', '12311'] # Replace with your actual IDRSSD listidrssd_list (List of banks you want to download reports)
while len(idrssd_list) > 0:
    time.sleep(2)

    # Re-create Chrome driver every time to avoid dead sessions
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--safebrowsing-disable-download-protection")
    chrome_options.add_argument("--user-data-dir=/tmp/temporary-profile")
    chrome_options.add_argument("window-size=1440,900")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "profile.default_content_settings.popups": 0,
        "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
    })

    driver = webdriver.Chrome(options=chrome_options)

    downloadRatio(idrssd_list, driver)  # pass the new driver each time
    print("Remaining banks:", len(idrssd_list))
print("Finishing collect all the data")



In [ ]:
import os
import pandas as pd
import glob

# Path to one CSV file
csv_folder = "/Users/kuangjinyan/Desktop/FUSE/call report clean/ruboteness_2" # Replace with your own path
all_files = glob.glob(os.path.join(csv_folder, "*.csv"))

merged_df = pd.DataFrame()
id_list = []
for file_path in all_files:

    idrssd = os.path.splitext(os.path.basename(file_path))[0]


    # Load CSV without headers
    df = pd.read_csv(file_path, header=None, usecols=[0, 1], names=["Date", "Value"])

    # Drop NaNs and garbage rows
    df = df.dropna(subset=["Date", "Value"])

    # Convert to datetime
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])

    # Convert Value column to numeric (just in case)
    df["Value"] = pd.to_numeric(df["Value"], errors="coerce")

    # Drop any remaining rows with invalid data
    df = df.dropna()

    # Pivot so that each column is a quarter, and row is the IDRSSD
    pivot_df = df.set_index("Date").T
    pivot_df.index = [idrssd]
    pivot_df.columns = pivot_df.columns.sort_values()

    merged_df = pd.concat([merged_df, pivot_df], axis=0)


merged_df.fillna(0)

# Load the mapping
map_df = pd.read_csv("UBPR_100billion_and_50billion_2024_12_31.csv", dtype=str) # Replance with your IDRSSD and Financial Instituition Name Maping list

# Create dictionary
id_to_name = dict(zip(map_df["IDRSSD"], map_df["Name"]))

# Replace index in merged_df
merged_df.rename(index=id_to_name, inplace=True)
merged_df.fillna(0)
if merged_df.index.name == "Date":
    print("'Date' was set as index name, correcting...")
    merged_df.index.name = None
merged_df.index.name = "Financial Institution Name"
print("Columns:", merged_df.columns.tolist())
print("Index name:", merged_df.index.name)
merged_df=merged_df.fillna(0)
merged_df